In [28]:
import requests
import time
import pickle
import os
import CF_standard_functions as cf_sf

In [29]:
# Set these parameters for saving and loading data
output_dir = os.getcwd()
file_name = 'CardData.pkl'
full_file = output_dir+"\\"+file_name

**Get Data from Scryfall**

https://scryfall.com/

For Scryfall we put the search query in as plain text as though it were the search
bar of the website. However we have to use some ASCII encoding of special 
characters (like =, <) as they have specific meanings used in an html string.
For example, %3D represents an equals sign , %3A represents a colon.
Look these up here: 

https://www.w3schools.com/tags/ref_urlencode.ASP

Separate clauses of the search are split with a plus sign.
You can use f-string interpolation to make it easier to edit the query.

In [30]:
# Scryfall query parameters:
order = "cmc"
# Unique printings of cards that have flavor text
querystring = "%28-is%3Afirstprint+-is%3Aunique%29+unique%3Aprints+has%3Aflavor&unique=cards"
url = f"https://api.scryfall.com/cards/search?order={order}&q={querystring}"

In [34]:
print(cf_sf.file_age_in_hours(full_file))
print(os.path.isfile(full_file))
print(cf_sf.file_age_in_hours(full_file))
print(len(data))

0.04193280637264252
True
0.04193280637264252
23365


In [35]:
# First check if data already exists and is "fresh". If so, load it in:
# Here defining fresh as newer than a week
if os.path.isfile(full_file) and cf_sf.file_age_in_hours(full_file) < 24*7 :
    print('Loading data from disk...')
    file = open(full_file, 'rb')
    card_data = pickle.load(file)
    file.close()
    print('Done!')
else:# Otherwise get the data from Scryfall. This can take a while 
    sf_response = requests.get(url).json()
    data = sf_response['data']
    curr_page = 1;
    print(f"Page {curr_page}",end='\r')
    while sf_response['has_more']:  
        time.sleep(0.2)
        url_next = sf_response['next_page']
        sf_response = requests.get(url_next).json()
        data_new = sf_response['data']
        data = data + data_new
        curr_page += 1 
        print(f"Page {curr_page}",end='\r')
    print(f"Loaded {curr_page} pages of cards")
    
    print("Saving...")
    file = open(full_file,'wb')    
    pickle.dump(data, file)
    file.close()
    print("Done!")
    
nCards = len(data)
print(nCards,"cards in data set")

Loading data from disk...
Done!
23365 cards in data set
